In [ ]:
%reload_azureml_ws

# 1. Azure Environments 
Python code runs in the context of a virtual environment that defines the version of the Python runtime to be used as well as the installed packages available to the code. In most Python installations, packages are installed and managed in environments using ***Conda*** or ***pip***.

## 1.1 Environments in Azure Machine Learning
In general, Azure Machine Learning handles environment creation and package installation for you - usually through the creation of Docker containers. You can specify the Conda or pip packages you need, and have Azure Machine Learning create an environment for the experiment.

In an enterprise machine learning solution, where experiments may be run in a variety of compute contexts, it can be important to be aware of the environments in which your experiment code is running. Environments are encapsulated by the Environment class; which you can use to create environments and specify runtime configuration for an experiment.

You can have Azure Machine Learning manage environment creation and package installation to define an environment, and then register it for reuse. Alternatively, you can manage your own environments and register them. This makes it possible to define consistent, reusable runtime contexts for your experiments - regardless of where the experiment script is run.

## 1.2 Creating Environments

There are a number of ways environment are created in the Azure Machine Learning, including when you:

- Initialize a new Environment object.

- Use one of the Environment class methods: 

    - [from_conda_specification(name, file_path)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py#from-conda-specification-name--file-path-),
    - [from_pip_requirements(name, file_path), or](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py#from-pip-requirements-name--file-path-)
    - [from_existing_conda_environment(name, conda_environment_name)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py#from-existing-conda-environment-name--conda-environment-name-).
    

- Use the submit(config, tags=None, **kwargs) method of the Experiment class to submit an experiment run without specifying an environment, including with an Estimator object.

> ####  ***Creating an Environment from a Specification File***
If you have an existing Conda environment defined on your workstation, you can use it to define an Azure Machine Learning environment:
```python
from azureml.core import Environment
env = Environment.from_existing_conda_environment(name='training_environment',
                                                  conda_environment_name='py_env')
```


> ####  ***Creating an Environment by Specifying Packages***
You can define an environment by specifying the Conda and pip packages you need in a CondaDependencies object, like this:
```python
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
env = Environment('training_environment')
deps = CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy'],
                                pip_packages=['azureml-defaults'])
env.python.conda_dependencies = deps
```

In the example below, we will create our azureDS environment from a ***requirements.txt***:
file

In [ ]:
from azureml.core import Environment

# Create a Python environment for the experiment
env = Environment.from_pip_requirements(name = 'azureDS',
                                        file_path = 'requirements.txt')

## 1.3 Register and Retrieve Environments

Having gone to the trouble of defining an environment with the packages you need, you can register it in the workspace.

Use the register method of an Environment object to register an environment:

In [ ]:
# Register the environment
env.register(workspace=ws)


Note that the environment is registered with the name you assigned when you first created it (in this case, *diabetes-experiment-env*).

With the environment registered, you can reuse it for any scripts that have the same requirements. 

You can view the registered environments in your workspace like this:

In [ ]:
env_names = Environment.list(workspace=ws)
for env_name in env_names:
    print('Name:',env_name)

# 1.4 Retrieving and using an Environment

You can retrieve a registered environment by using the get method of the Environment class, and then assign it to a ScriptRunConfig or Estimator.

For example, the following code sample retrieves the training_environment registered environment, and assigns it to an estimator:

```python
from azureml.core import Environment, Estimator

training_env = Environment.get(workspace=ws, name='training_environment')
estimator = Estimator(source_directory='experiment_folder'
                      entry_script='training_script.py',
                      compute_target='local',
                      environment_definition=training_env)
```
When an experiment based on the estimator is run, Azure Machine Learning will look for an existing environment that matches the definition, and if none is found a new environment will be created based on the registered environment specification.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment, Experiment
from azureml.widgets import RunDetails
from azureml.core import Dataset

# Get Environment 
training_env = Environment.get(workspace=ws, name='azureDS')

# Get a dataset reference from the workspace datasets collection
dataset = Dataset.get_by_name(ws, name='Iris Dataset')


# Create an estimator
estimator = Estimator(source_directory='src/azureds_util',
                      inputs=[dataset.as_named_input('iris')],
                      entry_script='run_azure.py',
                      compute_target='local',
                      environment_definition=training_env)

# Create an experiment
experiment = Experiment(workspace = ws, name = 'Env_trial')

# Run the experiment
run = experiment.submit(config=estimator)

# Show the run details while running
RunDetails(run).show()
run.wait_for_completion()

# 1.5 Curated Environments

Azure Machine Learning includes a selection of pre-defined curated environments that reflect common usage scenarios. These include environments that are pre-configured with package dependencies for common frameworks, such as Scikit-Learn, PyTorch, Tensorflow, and others.

Curated environments are registered in all Azure Machine Learning workspaces with a name that begins ***AzureML-***.

>***Note***: You can't register your own environments with an “AzureML-” prefix.

To view curated environments and the dependencies they contain, you can run the following code:



In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)
for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

# 2. Compute Targets

In Azure Machine Learning, [Compute](https://docs.microsoft.com/en-gb/azure/machine-learning/concept-compute-target) Targets are physical or virtual computers on which experiments are run.

### Flexible Compute
The ability to assign experiment runs to specific compute targets helps you implement a flexible data science ecosystem in the following ways:

- Code can be developed and tested on local or low-cost compute, and then moved to more scalable compute for production workloads.
- You can run individual processes on the compute target that best fits its needs. For example, by using GPU-based compute to train deep learning models, and switching to lower-cost CPU-only compute to test and register the trained model.

### Cost-Effective Scalability
One of the core benefits of cloud computing is the ability to manage costs by paying only for what you use. In Azure Machine Learning, you can take advantage of this principle by defining compute targets that:

- Start on-demand and stop automatically when no longer required.
- Scale automatically based on workload processing needs.


## 2.1 Types of Compute
Azure Machine Learning supports multiple types of compute for experimentation and training, and for production inferencing. This enables you to select the most appropriate type of compute target for your particular needs.

>### 2.1.1 Local Compute
You can specify a local compute target for most processing tasks in Azure Machine Learning. This runs the experiment on the same compute target as the code used to initiate the experiment, which may be you physical workstation or a virtual machine such as an Azure Machine Learning compute instance on which you are running a notebook.
Local compute is generally a great choice during development and testing with low to moderate volumes of data.
### 2.1.2 Training Clusters
For training workloads with high scalability requirements, you can use Azure Machine Learning training clusters; which are multi-node clusters of Virtual Machines that automatically scale up or down to meet demand. This is a cost-effective way to run experiments that need to handle large volumes of data or use parallel processing to distribute the workload and reduce the time it takes to run. <br><br>
***Note***: Despite the name, you can also use training clusters to deploy batch inferencing pipelines in production. You'll explore this further later in the course.
### 2.1.3 Inference Clusters
To deploy trained models as production services, you can use Azure Machine Learning inference clusters, which use containerization technologies to enable rapid initialization of compute for on-demand inferencing.
### 2.1.4 Attached Compute
If you already use an Azure-based compute environment for data science, such as a virtual machine or an Azure Databricks cluster, you can attach it to your Azure Machine Learning workspace and use it as a compute target for certain types of workload.

## 2.2 Create a Compute Target
In many cases, your local compute resources may not be sufficient to process a complex or long-running experiment that needs to process a large volume of data; and you may want to take advantage of the ability to dynamically create and use compute resources in the cloud.
Azure ML supports a range of compute targets, which you can define in your workpace and use to run experiments; paying for the resources only when using them. When you set up the workspace in the first exercise, you created a training cluster called **aml-cluster**, so let's verify that exists (and if not, create it) so we can use it to run training experiments.


In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aml-cluster"

# Verify that cluster exists
try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS1_V2', max_nodes=2)
    training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

training_cluster.wait_for_completion(show_output=True)

#### ***Creating a Managed Compute Target with the SDK***

A managed compute target is one that is managed by Azure Machine Learning, such as an Azure Machine Learning training cluster.

To create an Azure Machine Learning training cluster compute target, use the azureml.core.compute.ComputeTarget class and the AmlCompute class, like this:

>```python
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
# Load the workspace from the saved config file
ws = Workspace.from_config()
# Specify a name for the compute (unique within the workspace)
compute_name = 'aml-cluster'
# Define compute configuration
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                       min_nodes=0, max_nodes=4,
                                                       vm_priority='dedicated')
# Create the compute
aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)
aml_cluster.wait_for_completion(show_output=True)
```

In this example, a cluster with up to four nodes based on the STANDARD_DS12_v2 virtual machine image will be created. The priority for the virtual machines (VMs) is set to dedicated, meaning they are reserved for use in this cluster (the alternative is to specify lowpriority, which has a lower cost but means that the VMs can be pre-empted if a higher-priority workload requires the compute).

Note: For a full list of AmlCompute configuration options, see the AmlCompute class SDK documentation.

#### ***Attaching an Unmanaged Compute Target with the SDK***

An unmanaged compute target is one that is defined and managed outside of the Azure Machine Learning workspace; for example, an Azure virtual machine or an Azure Databricks cluster.

The code to attach an existing unmanaged compute target is similar to the code used to create a managed compute target, except that you must use the ComputeTarget.attach() method to attach the existing compute based on its target-specific configuration settings.

For example, the following code can be used to attach an existing Azure Databricks cluster:

>```python
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, DatabricksCompute
# Load the workspace from the saved config file
ws = Workspace.from_config()
# Specify a name for the compute (unique within the workspace)
compute_name = 'db_cluster'
# Define configuration for existing Azure Databricks cluster
db_workspace_name = 'db_workspace'
db_resource_group = 'db_resource_group'
db_access_token = '1234-abc-5678-defg-90...'
db_config = DatabricksCompute.attach_configuration(resource_group=db_resource_group,
                                                   workspace_name=db_workspace_name,
                                                   access_token=db_access_token)
# Create the compute
databricks_compute = ComputeTarget.attach(ws, compute_name, db_config)
databricks_compute.wait_for_completion(True)
```

#### ***Checking for an Existing Compute Target***


In many cases, you will want to check for the existence of a compute target, and only create a new one if there isn't already one with the specified name. To accomplish this, you can catch the ComputeTargetException exception, like this:

>```python
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
compute_name = "aml-cluster"
# Check if the compute target exists
try:
    aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)
aml_cluster.wait_for_completion(show_output=True)
```


More information for creating compute targets can be found here:

https://docs.microsoft.com/en-gb/azure/machine-learning/how-to-set-up-training-targets